### Dynamic Programming with shortage cost 

#### How to run each test instance


**Instance 1:**  

    1) 
    
    2) Click 'Kernel' in the menu and select 'Restart & Run All' to run the program once.

**Instance 2:** 
    
     1) 
    
     2) Click 'Kernel' in the menu and select 'Restart & Run All' to run the program once.

**Instance 3:** 
    
     1) 
    
     2) Click 'Kernel' in the menu and select 'Restart & Run All' to run the program once.

#### Import Modules

In [ ]:
# reading file
import pandas as pd

#math and random operations
import random as rd
import math
import numpy as np

# Generating graphs
import matplotlib.pyplot as plt; plt.rcdefaults()
import numpy as np
import matplotlib.pyplot as plt

#### Define DP Stages and States and variables

In [ ]:
rd.seed(2844)
T_1,T_2,T_3 = 10, 20, 30
S = 40
Q = 40
I0 = 0
ct = round(rd.uniform(10,20),1)
ht = round(rd.uniform(5,15),1)
st_1 = round(rd.uniform(15,35),1) 
st_2 = round(rd.uniform(10,25),1) 
st_3 = round(rd.uniform(5,15),1) 

#### Helper functions

In [ ]:
def dummy_fill_In(table, tablenum):
    if (tablenum == 1):
        table.append([i for i in range(0,Q)])
    else:    
        for row in range(0,S):
            list = []
            for col in range(0,Q):
                list.append(col)
            table.append(list)
        
def fillIn_StartBoundary(table, tablenum, dt, ht, ct, st_1):
    for col in range(0, len(table[tablenum][0])):
        nextI = max(0, I0 + col - dt)
        shortI = max(0, dt-I0-col)
        if(nextI > 0):
            cost = round(ct*col + ht*nextI + st_1*shortI + OptimalDec(table, 2, nextI, dt),1)
            table[tablenum][0][col] = cost
        else:
            cost = round(ct*col + ht*nextI + st_1*shortI + OptimalDec(table, 2, 0, dt),1)
            table[tablenum][0][col] = cost
    return table

def fillIn_EndBoundary(table, tablenum, dt, ct, st_1):
    for row in range(0, len(table[tablenum])):
        for col in range(0, len(table[tablenum][row])):
            nextI = max(0, row + col - dt)
            shortI = max(0, dt-row-col)
            if(nextI > 0):
                cost = round(ct*col + ht*nextI + st_1*shortI, 1)
                table[tablenum][row][col] = cost
            else:
                cost = round(ct*col + ht*nextI + st_1*shortI, 1)
                table[tablenum][row][col] = cost
    return table

def fillIn_Table(table, tablenum, dt, da, ht, ct, st_1):
    for row in range(0, len(table[tablenum])):
        for col in range(0, len(table[tablenum][row])):
            nextI = max(0, row + col - dt)
            if(nextI >= S):
                nextI = S-1
            shortI = max(0, dt-row-col)
            if(shortI>= S):
                shortI = S-1
            nexttable = tablenum + 1
            if(nextI > 0):
                cost = round(ct*col + ht*nextI + st_1*shortI + OptimalDec(table, nexttable, nextI, dt),1)
                table[tablenum][row][col] = cost
            else:
                cost = round(ct*col + ht*nextI+ st_1*shortI + OptimalDec(table, nexttable, 0, dt),1)
                table[tablenum][row][col] = cost
#                 print(f'SHORTING table={tablenum},ht={ht}, ct={ct}, st={st_1} dt={dt}, da={da}, row={row}, col={col}, nexttable={nexttable}, nextI={nextI}, shortI={shortI}, cost={cost}')
    return table

def OptimalDec(table, tablenum, row, dt):
    opt = 999
    optindex = -1
    for col in range(0,len(table[tablenum][row])):
        if(table[tablenum][row][col] <= opt):
            optindex = col
            opt = table[tablenum][row][col]
    return opt

def findMinInRowIndex(table, tablenum, row, dt):
    FinalCost = 99999
    indexOfFinalCost = 0
    for col in range(0,len(table[tablenum][row])):
        if(table[tablenum][row][col] <= FinalCost):
            FinalCost = table[tablenum][row][col]
            indexOfFinalCost = col + 1 
    return indexOfFinalCost

def recursive(table, tablenum, dt, ht, ct, dts):
    if(tablenum == 1):
#         print(f'tablenum={tablenum}, dt={dt}')
        dts.append(dt)
        table = fillIn_StartBoundary(table, 1, dt, ht, ct, st_1)
        return table
    else:
#         print(f'tablenum={tablenum}, dt={dt}')
        dts.append(dt)
        if(tablenum == T_1):
#             print(f'tablenum={tablenum}, dt={dt}')
            table = fillIn_EndBoundary(table, 10, dt, ct, st_1)
            dt = int(rd.uniform(0,20))
            tablenum = tablenum - 1
            return recursive(table, tablenum, dt, ht, ct, dts)
        else:
            da = int(rd.uniform(0,20))
            table = fillIn_Table(table, tablenum, dt, da, ht, ct, st_1)
            dt = da
            tablenum = tablenum - 1
            return recursive(table, tablenum, da, ht, ct, dts)

#### Solve and Run DP

In [ ]:
%%time
rd.seed(2844)
dts = []

def Run():
    Case_1 ={1:[],2:[],3:[],4:[],5:[],6:[],7:[],8:[],9:[],10:[]} 
    for i in Case_1:
        dummy_fill_In(Case_1[i], i)

    dt = int(rd.uniform(0,20))
    Case_1 = recursive(Case_1, T_1, dt, ht, ct, dts)
    return Case_1

# Run
table = Run() 

n = len(dts)-1
li = []
CurrInv = I0
NextInv = 0 
ShortInv = 0
for t in range(1,11):
    if(t == 1):
        xVal = findMinInRowIndex(table, t, CurrInv, dts[n])
        li.append([CurrInv, ShortInv, xVal, dts[n]])
        NextInv = max(0, I0 + xVal - dts[n])
        if(NextInv >= S):
            NextInv = S-1
        ShortInv = max(0, dts[n]- I0 - xVal)
        if(ShortInv >= S):
            ShortInv = S-1
        if(NextInv > 0):
            NextInv = NextInv
        else:
            NextInv = 0
#         print(f'table={t}, CurrInv={CurrInv}, NextInv={NextInv}, ShortI={ShortInv}, Col={xVal}, dts={dts[n]}')
    else:
        if(n == 0):
            xVal = findMinInRowIndex(table, t, CurrInv, dts[n])
            li.append([CurrInv, ShortInv, xVal, dts[n]])
            if(NextInv >= S):
                NextInv = S-1
            ShortInv = max(0, dts[n]- CurrInv - xVal)
            if(ShortInv >= S):
                ShortInv = S-1
            if(NextInv > 0):
                NextInv = NextInv
            else:
                NextInv = 0
#             print(f'table={t}, CurrInv={CurrInv}, NextInv={NextInv}, ShortI={ShortInv}, Col={xVal}, dts={dts[n]}')
        else:
            n = n-1
            CurrInv = NextInv
            if(CurrInv >= S):
                CurrInv = S-1
            xVal = findMinInRowIndex(table, t, CurrInv, dts[n])
            ShortInv = max(0, dts[n]- CurrInv - xVal)
            li.append([CurrInv, ShortInv, xVal, dts[n]])
            NextInv = max(0, CurrInv + xVal - dts[n])
            if(NextInv >= S):
                NextInv = S-1
            if(ShortInv >= S):
                ShortInv = S-1
            if(NextInv > 0):
                NextInv = NextInv
            else:
                NextInv = 0
#             print(f'table={t}, CurrInv={CurrInv}, NextInv={NextInv}, ShortI={ShortInv}, Col={xVal}, dts={dts[n]}')

# print(li,'\n', ct, ht, st_1,'\n')

# Generate Average + Standard Deviation over all the 10 instances, prodcution cost, holding cost, runtime
totCost = []
prodCost = []
holdCost = []
ShortCost = []
tables = []
AverageTotalCost = []

CurrentInv= []
ShortInv = []
UnitsProd = []
Demand = []
for i in range(0, len(li)):
    CurrentInv.append(li[i][0])
    ShortInv.append(li[i][1])
    UnitsProd.append(li[i][2])
    Demand.append(li[i][3])

for i in range(1, 11):
    tables.append(i)
    pc = round(ct*li[i-1][2],1)
    prodCost.append(pc)
    hc = round(ht*(max(0,(li[i-1][0]+li[i-1][2]-li[i-1][3]))),1)
    holdCost.append(hc)
    sc = round(st_1*(max(0,(li[i-1][3]-li[i-1][0]-li[i-1][2]))),1)
    ShortCost.append(sc)
    totCost.append(pc + hc + sc)
df = pd.DataFrame({
        'Period':tables,
        'Current Inventory':CurrentInv,
        'Units Produced':UnitsProd,
        'Demand':Demand,
        'Unsatisfied Demand':ShortInv,
        'Production Cost' : prodCost,
        'Holding Cost' : holdCost,
        'Shortage Cost':ShortCost,
        'Total Cost In Period' : totCost
    })

SumCost = df.sum(axis=0)
AvgProdCost = df.mean(axis=0)
df

#### Get Averages and Total Cost

In [ ]:
print(f'Average Production Cost = {round(AvgProdCost[5],1)} \nTotal Cost = {round(SumCost[8],1)}')

#### Test Cases